# Get Origin Block Groups from Patterns

White Pass Ski area, when you ride up on the lift it is common for people to hail from Yakima, the Tri-Cities, Portland-Vancouver, Olympia and Tacoma. White Pass is well known for being a very low-key, friendly and extremely unpretentious ski area. With Mt. Hood near Portland and Stephens Pass and Crystal near Seattle getting increasingly crowded, there is a belief more people are beginning to think the drive to White Pass is worth it. Talking to people on the lift, though, is little more than antecdoctal. 

Where are people _really_ coming from, and are these patterns changing? We can investigate these trends using Safegraph Patterns data.

In [1]:
import importlib
import json
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd

In [2]:
dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'

dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'

gdb_raw = dir_raw/'raw.geodatabase'
gdb_int = dir_int/'interim.geodatabase'

Using the previously downloaded patterns data for White Pass, the first step is simply loading the data into a Pandas DataFrame. This DataFrame contains quite a few columns encoded as dictionaries, which can be expanded for more data detail.

In [3]:
wp_pth = dir_raw/'patterns_wp.csv'

In [4]:
wp_df = pd.read_csv(wp_pth, index_col=0)

wp_df.head()

,safegraph_place_id,location_name,street_address,city,region,postal_code,safegraph_brand_ids,brands,date_range_start,date_range_end,...,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,placekey,parent_placekey,parent_safegraph_place_id
137646,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2020-05-01T00:00:00-07:00,2020-06-01T00:00:00-07:00,...,21.0,"{""<5"":3,""5-20"":56,""21-60"":26,""61-240"":27,"">240...","{""Fred Meyer Jewelers"":33,""Chevron"":32}","{""Chevron"":37,""Walmart"":24,""76"":23,""Safeway"":2...","[5,4,3,2,4,4,4,12,21,28,27,26,23,22,26,22,9,11...","{""Monday"":11,""Tuesday"":20,""Wednesday"":15,""Thur...","{""android"":44,""ios"":32}",NaN,NaN,NaN
199898,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2020-06-01T00:00:00-07:00,2020-07-01T00:00:00-07:00,...,35.0,"{""<5"":1,""5-20"":17,""21-60"":7,""61-240"":18,"">240"":2}",{},"{""Chevron"":34,""Shell Oil"":34,""Safeway Fuel Sta...","[0,1,2,2,2,1,1,10,10,11,11,11,5,12,12,10,2,1,0...","{""Monday"":11,""Tuesday"":7,""Wednesday"":5,""Thursd...","{""android"":13,""ios"":7}",NaN,NaN,NaN
198080,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2020-07-01T00:00:00-07:00,2020-08-01T00:00:00-07:00,...,24.0,"{""<5"":3,""5-20"":33,""21-60"":22,""61-240"":12,"">240...",{},"{""Chevron"":43,""Walmart"":29,""76"":26,""Shell Oil""...","[2,2,2,2,4,3,1,6,9,8,13,15,25,23,22,15,9,3,3,6...","{""Monday"":10,""Tuesday"":10,""Wednesday"":11,""Thur...","{""android"":38,""ios"":21}",NaN,NaN,NaN
137469,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2020-08-01T00:00:00-07:00,2020-09-01T00:00:00-07:00,...,22.0,"{""<5"":3,""5-20"":37,""21-60"":24,""61-240"":13,"">240...",{},"{""Chevron"":42,""Shell Oil"":32,""Safeway"":28,""76""...","[2,2,2,2,3,2,4,20,17,20,16,21,22,21,17,16,9,4,...","{""Monday"":19,""Tuesday"":15,""Wednesday"":10,""Thur...","{""android"":39,""ios"":17}",NaN,NaN,NaN
86499,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,...,52.5,"{""<5"":1,""5-20"":34,""21-60"":32,""61-240"":23,"">240...",{},"{""Costco Wholesale Corp."":31,""76"":29,""Shell Oi...","[13,14,11,12,12,13,21,31,28,28,31,27,28,32,29,...","{""Monday"":21,""Tuesday"":10,""Wednesday"":31,""Thur...","{""android"":31,""ios"":14}",zzy-222@5xd-7jh-f75,NaN,NaN


One of these columns `visitor_home_cbgs` is a dictionary of Census Block Groups with the count of unique devices originating in the block group visiting the point of interest, in this case White Pass Ski Area, in each of the months. Since this column is read in as a string from the CSV, we are going to convert it to a dictionary using `json.loads`.

In [5]:
wp_df.visitor_home_cbgs = wp_df.visitor_home_cbgs.apply(lambda val: json.loads(val))

wp_df.visitor_home_cbgs

137646    {'530419717002': 5, '530770028012': 4, '530670...
199898    {'410099706003': 4, '530150015012': 4, '530770...
198080    {'530419715002': 5, '410510104091': 4, '530459...
137469    {'530150018002': 4, '530419720004': 4, '530699...
86499     {'530419715003': 4, '530770031003': 4, '530210...
169339    {'530730011001': 4, '530770028012': 4, '530350...
233622    {'530530724102': 7, '530530715031': 6, '530670...
94681     {'530670117102': 10, '530770034001': 7, '53015...
128545    {'530530724051': 10, '530530724063': 7, '53067...
119562    {'530499505002': 8, '530770032001': 7, '530770...
143458    {'530770030022': 8, '530770028012': 5, '530670...
184700    {'530770022001': 4, '530530605001': 4, '530770...
191186    {'530770009014': 5, '530210208004': 4, '530530...
226235    {'530630131004': 4, '530770031001': 4, '171790...
31424     {'530419720003': 8, '530419718002': 5, '530110...
189140    {'530050119003': 5, '530050115042': 4, '530670...
197813    {'530770031003': 4, '530050107

The ski season generally runs November through March. Although we have data for the entire year, we are only going to focus on the months comprising the ski season. To do this, we need to extract the year and month explicitly from a datetime field. From there, we can filter to just the months we are interested in.

In [6]:
mnths = [11, 12, 1, 2, 3, 4, 5]

wp_df['year'] = pd.to_datetime(wp_df.date_range_start).apply(lambda dt: dt.year)
wp_df['month'] = pd.to_datetime(wp_df.date_range_start).apply(lambda dt: dt.month)

dict_df = wp_df[wp_df.month.isin(mnths)][['year', 'month', 'visitor_home_cbgs']].sort_values(['year', 'month'])

Since, at least in this case, we are interested in trends for each ski season as a whole, we are also going to create a label we can later use to bin the data.

In [7]:
dict_df['season'] = dict_df[['year', 'month']].apply(lambda r: f'{r.year-1}_{r.year}' if r.month < 6 else f'{r.year}_{r.year+1}', axis=1)

dict_df

,year,month,visitor_home_cbgs,season
94681,2018,1,"{'530670117102': 10, '530770034001': 7, '53015...",2017_2018
128545,2018,2,"{'530530724051': 10, '530530724063': 7, '53067...",2017_2018
119562,2018,3,"{'530499505002': 8, '530770032001': 7, '530770...",2017_2018
143458,2018,4,"{'530770030022': 8, '530770028012': 5, '530670...",2017_2018
184700,2018,5,"{'530770022001': 4, '530530605001': 4, '530770...",2017_2018
168211,2018,11,"{'530670127301': 5, '530419720003': 4, '410459...",2018_2019
228611,2018,12,"{'530770028012': 11, '530419720003': 10, '5306...",2018_2019
178995,2019,1,"{'530419720003': 11, '530670117103': 9, '53005...",2018_2019
158825,2019,2,"{'530530712062': 9, '530770028012': 8, '530670...",2018_2019
137495,2019,3,"{'530419720003': 11, '530770008001': 7, '53077...",2018_2019


Next, although a trival step, we reduce the dataframe down to just the columns we need, the binning season and the dictionary of home block groups and device counts.

In [8]:
dict_df = dict_df[['season', 'visitor_home_cbgs']]

dict_df

,season,visitor_home_cbgs
94681,2017_2018,"{'530670117102': 10, '530770034001': 7, '53015..."
128545,2017_2018,"{'530530724051': 10, '530530724063': 7, '53067..."
119562,2017_2018,"{'530499505002': 8, '530770032001': 7, '530770..."
143458,2017_2018,"{'530770030022': 8, '530770028012': 5, '530670..."
184700,2017_2018,"{'530770022001': 4, '530530605001': 4, '530770..."
168211,2018_2019,"{'530670127301': 5, '530419720003': 4, '410459..."
228611,2018_2019,"{'530770028012': 11, '530419720003': 10, '5306..."
178995,2018_2019,"{'530419720003': 11, '530670117103': 9, '53005..."
158825,2018_2019,"{'530530712062': 9, '530770028012': 8, '530670..."
137495,2018_2019,"{'530419720003': 11, '530770008001': 7, '53077..."


Next, we explode out the all the values from the dictionary to be columns in the dataframe.

In [9]:
bg_df = dict_df[['season']].join(dict_df.visitor_home_cbgs.apply(pd.Series))

bg_df

,season,530670117102,530770034001,530150009003,530770008001,530050108033,530459613002,530530728001,350579632021,530419720003,...,530530725041,530350928023,530210206011,530050102021,530530703093,530050109014,530530604003,530530623002,530459605003,530330082002
94681,2017_2018,10.0,7.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128545,2017_2018,4.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119562,2017_2018,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143458,2017_2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184700,2017_2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168211,2018_2019,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228611,2018_2019,4.0,4.0,NaN,NaN,5.0,NaN,4.0,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178995,2018_2019,NaN,4.0,NaN,4.0,4.0,4.0,7.0,NaN,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158825,2018_2019,NaN,5.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137495,2018_2019,NaN,4.0,NaN,7.0,NaN,4.0,4.0,NaN,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we can simply use `groupby` and `max` to get the maximum number of devices per month originating from each block group for each season. Also, while we are at it, we can fill in any missing values with zeros.

In [10]:
bg_df = bg_df.groupby('season').max().fillna(0)

bg_df

,530670117102,530770034001,530150009003,530770008001,530050108033,530459613002,530530728001,350579632021,530419720003,530530606001,...,530530725041,530350928023,530210206011,530050102021,530530703093,530050109014,530530604003,530530623002,530459605003,530330082002
season,,,,,,,,,,,,,,,,,,,,,
2017_2018,10.0,7.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018_2019,4.0,5.0,0.0,7.0,5.0,4.0,7.0,0.0,11.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019_2020,7.0,9.0,0.0,9.0,6.0,0.0,4.0,0.0,12.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020_2021,0.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


With all this organized, now we can organize the data by the block group id using `transpose`.

In [11]:
bg_df.transpose()

season,2017_2018,2018_2019,2019_2020,2020_2021
530670117102,10.0,4.0,7.0,0.0
530770034001,7.0,5.0,9.0,4.0
530150009003,6.0,0.0,0.0,0.0
530770008001,6.0,7.0,9.0,4.0
530050108033,6.0,5.0,6.0,4.0
...,...,...,...,...
530050109014,0.0,0.0,0.0,4.0
530530604003,0.0,0.0,0.0,4.0
530530623002,0.0,0.0,0.0,4.0
530459605003,0.0,0.0,0.0,4.0
